In [ ]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import numpy as np
import gc
import os


In [30]:
gc.collect()
df_full = pd.read_parquet('./data/l_vm_completa_train.parquet', engine='fastparquet')

In [31]:
# Optimizar tipos de datos numéricos
for col in df_full.select_dtypes(include=['int64']).columns:
    df_full[col] = pd.to_numeric(df_full[col], downcast='integer')
for col in df_full.select_dtypes(include=['float64']).columns:
    df_full[col] = pd.to_numeric(df_full[col], downcast='float')
# Variables categóricas
# categorical_features = ['ANIO','MES','TRIMESTRE','ID_CAT1','ID_CAT2','ID_CAT3','ID_BRAND','SKU_SIZE','CUSTOMER_ID','PRODUCT_ID','PLAN_PRECIOS_CUIDADOS']
categorical_features = ['ID_CAT1','ID_CAT2','ID_CAT3','ID_BRAND','PLAN_PRECIOS_CUIDADOS']
# Convertir las variables categóricas a tipo 'category'
for col in categorical_features:
    df_full[col] = df_full[col].astype('category')

In [32]:
# Variables predictoras y objetivo
# filtrar que en X el periodo sea menor o igual a 201910
# En x eliminar la columna 'CLASE' y 'CLASE_DELTA'
X = df_full[df_full['PERIODO'] <= 201910].drop(columns=['CLASE', 'CLASE_DELTA']) 
# Filtrar en y que el periodo sea menor o igual a 201910
y = df_full[df_full['PERIODO'] <= 201910]['CLASE']
# Agrega una columna 'CLASE_BIN' que sea 1 si 'CLASE' es mayor a 0, y 0 en caso contrario para clasificación binaria
# El tipo es cero o uno por lo que se convierte a int del menor tamaño posible
df_full['CLASE_BIN'] = (df_full['CLASE'] > 0).astype('int8')
y_bin = df_full[df_full['PERIODO'] <= 201910]['CLASE_BIN']

# Eliminar df_full para liberar memoria
del df_full
gc.collect()

0

In [33]:
# Definir los periodos de validación 201910
periodos_valid = [201910]

# Separar train y cinco conjuntos de validación respetando la secuencia temporal
X_train = X[X['PERIODO'] < periodos_valid[0]]
y_train = y[X['PERIODO'] < periodos_valid[0]]
y_bin_train = y_bin[X['PERIODO'] < periodos_valid[0]]

X_val_list = [X[X['PERIODO'] == p] for p in periodos_valid]
y_val_list = [y[X['PERIODO'] == p] for p in periodos_valid]
y_bin_val_list = [y_bin[X['PERIODO'] == p] for p in periodos_valid]

del X, y, y_bin
gc.collect()

0

In [46]:
import optuna

def objective(trial):
    params = {
        'objective': 'regression',
        'metric': [],  # Solo métrica personalizada
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 16, 1023),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
        'verbose': -1
    }

    model = lgb.train(
        params,
        train_data,
        num_boost_round=5000,
        valid_sets=val_data_list,
        valid_names=[f'validation_{p}' for p in periodos_valid],
        feval=mape_sum_lgb,
        callbacks=[
            lgb.early_stopping(stopping_rounds=50, first_metric_only=True),
            lgb.log_evaluation(period=0)
        ]
    )
    # Optuna minimiza, así que retornamos el mejor valor de la métrica personalizada
    best_score = model.best_score[f'validation_{periodos_valid[0]}']['mape_sum']
    return best_score

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)  # Puedes aumentar n_trials para mejor resultado

print("Mejores hiperparámetros encontrados:")
print(study.best_params)

[I 2025-06-11 00:51:24,495] A new study created in memory with name: no-name-6f53362f-a19a-4d6e-988c-2650213f2831


Training until validation scores don't improve for 50 rounds


[I 2025-06-11 00:51:48,199] Trial 0 finished with value: 0.41790189960396257 and parameters: {'num_leaves': 179, 'learning_rate': 0.015497518544334897, 'feature_fraction': 0.9665555850535374, 'bagging_fraction': 0.5734231892248185, 'bagging_freq': 9, 'min_data_in_leaf': 63}. Best is trial 0 with value: 0.41790189960396257.


Early stopping, best iteration is:
[128]	validation_201910's l2: 0.609082	validation_201910's mape_sum: 0.417902
Evaluated only: l2
Training until validation scores don't improve for 50 rounds


[I 2025-06-11 00:51:58,441] Trial 1 finished with value: 0.3860638429285289 and parameters: {'num_leaves': 96, 'learning_rate': 0.09563784387578975, 'feature_fraction': 0.7570492321866702, 'bagging_fraction': 0.6430771145469985, 'bagging_freq': 3, 'min_data_in_leaf': 46}. Best is trial 1 with value: 0.3860638429285289.


Early stopping, best iteration is:
[24]	validation_201910's l2: 0.59317	validation_201910's mape_sum: 0.386064
Evaluated only: l2
Training until validation scores don't improve for 50 rounds


[W 2025-06-11 00:52:16,746] Trial 2 failed with parameters: {'num_leaves': 934, 'learning_rate': 0.04766456491136605, 'feature_fraction': 0.5274180544739613, 'bagging_fraction': 0.9054890760351441, 'bagging_freq': 9, 'min_data_in_leaf': 25} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/pablo/anaconda3/envs/LaboIII/lib/python3.9/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_17869/38763556.py", line 17, in objective
    model = lgb.train(
  File "/home/pablo/anaconda3/envs/LaboIII/lib/python3.9/site-packages/lightgbm/engine.py", line 322, in train
    booster.update(fobj=fobj)
  File "/home/pablo/anaconda3/envs/LaboIII/lib/python3.9/site-packages/lightgbm/basic.py", line 4111, in update
    _LIB.LGBM_BoosterUpdateOneIter(
KeyboardInterrupt
[W 2025-06-11 00:52:16,748] Trial 2 failed with value None.


KeyboardInterrupt: 

In [ ]:

model_reg = None
# # Hacer un try-except para cargar el modelo de LightGBM para regresión
# try:
#     # Verificar si el archivo del modelo de regresión existe
#     if not os.path.exists('./modelos/lgbm_model_reg.txt'):
#         raise FileNotFoundError("El modelo de regresión no se encuentra en la ruta especificada.")
#     model_reg = lgb.Booster(model_file='./modelos/lgbm_model_reg.txt')
#     print("Modelo de regresión cargado exitosamente.")
# except FileNotFoundError:
#     model_reg = None

if model_reg is None:
    # Crear los datasets de LightGBM
    train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=categorical_features)
    val_data_list = []
    for i in range(len(periodos_valid)):
        val_dataset = lgb.Dataset(X_val_list[i], label=y_val_list[i], categorical_feature=categorical_features)
        # Inyectar PRODUCT_ID como atributo extra
        val_dataset.PRODUCT_ID = X_val_list[i]['PRODUCT_ID'].values
        val_data_list.append(val_dataset)

    def mape_sum_lgb(y_pred, dataset):
        y_true = dataset.get_label()
        product_id = getattr(dataset, 'PRODUCT_ID', None)
        y_pred = np.where(y_pred < 0, 0, y_pred)
        denom = np.sum(np.abs(y_true))
        if denom == 0:
            return 'mape_sum', 0.0, False
        if product_id is not None:
            df_pred = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred, 'PRODUCT_ID': product_id})
            mape = np.sum(np.abs(df_pred.groupby('PRODUCT_ID')['y_true'].sum() - df_pred.groupby('PRODUCT_ID')['y_pred'].sum())) / denom
        else:
            mape = np.sum(np.abs(y_true.sum() - y_pred.sum())) / denom
        mape = np.nan_to_num(mape, nan=0.0)
        return 'mape_sum', mape, False  # False: menor es mejor

    # Definir parámetros para regresión
    params = { 
        'objective': 'regression',
        'metric': 'None',  # <--- Muy importante: dejarlo VACÍO para usar solo la métrica custom
        'boosting_type': 'gbdt',
        'num_leaves': 511,
        'learning_rate': 0.005,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': -1
    }

    # Entrenar el modelo con la métrica personalizada
    model_reg = lgb.train(
        params,
        train_data,
        num_boost_round=50000,
        valid_sets=val_data_list,
        valid_names=[f'validation_{p}' for p in periodos_valid],
        feval=mape_sum_lgb,
        callbacks=[
            lgb.early_stopping(stopping_rounds=1000, first_metric_only=True),  # <--- Solo la primera métrica custom
            lgb.log_evaluation(period=50)
        ]
    )

    # Guardar el modelo entrenado
    os.makedirs('./modelos', exist_ok=True)
    model_reg.save_model('./modelos/lgbm_model_reg.txt')

Training until validation scores don't improve for 1000 rounds
[50]	validation_201910's mape_sum: 1.39152
[100]	validation_201910's mape_sum: 1.10548
[150]	validation_201910's mape_sum: 0.894324
[200]	validation_201910's mape_sum: 0.737199
[250]	validation_201910's mape_sum: 0.620146
[300]	validation_201910's mape_sum: 0.53026
[350]	validation_201910's mape_sum: 0.461553
[400]	validation_201910's mape_sum: 0.410732
[450]	validation_201910's mape_sum: 0.376511


KeyboardInterrupt: 

In [ ]:
import optuna

# Crear los datasets de LightGBM
train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=categorical_features)
val_data_list = []
for i in range(len(periodos_valid)):
    val_dataset = lgb.Dataset(X_val_list[i], label=y_val_list[i], categorical_feature=categorical_features)
    # Inyectar PRODUCT_ID como atributo extra
    val_dataset.PRODUCT_ID = X_val_list[i]['PRODUCT_ID'].values
    val_data_list.append(val_dataset)

def mape_sum_lgb(y_pred, dataset):
    y_true = dataset.get_label()
    product_id = getattr(dataset, 'PRODUCT_ID', None)
    y_pred = np.where(y_pred < 0, 0, y_pred)
    denom = np.sum(np.abs(y_true))
    if denom == 0:
        return 'mape_sum', 0.0, False
    if product_id is not None:
        df_pred = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred, 'PRODUCT_ID': product_id})
        mape = np.sum(np.abs(df_pred.groupby('PRODUCT_ID')['y_true'].sum() - df_pred.groupby('PRODUCT_ID')['y_pred'].sum())) / denom
    else:
        mape = np.sum(np.abs(y_true.sum() - y_pred.sum())) / denom
    mape = np.nan_to_num(mape, nan=0.0)
    return 'mape_sum', mape, False  # False: menor es mejor

def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'None',  # Solo métrica personalizada
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 16, 1023),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
        'verbose': -1
    }

    model = lgb.train(
        params,
        train_data,
        num_boost_round=5000,
        valid_sets=val_data_list,
        valid_names=[f'validation_{p}' for p in periodos_valid],
        feval=mape_sum_lgb,
        callbacks=[
            lgb.early_stopping(stopping_rounds=50, first_metric_only=True),
            lgb.log_evaluation(period=0)
        ]
    )
    # Optuna minimiza, así que retornamos el mejor valor de la métrica personalizada
    best_score = model.best_score[f'validation_{periodos_valid[0]}']['mape_sum']
    return best_score

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)  # Puedes aumentar n_trials para mejor resultado

print("Mejores hiperparámetros encontrados:")
print(study.best_params)

# Entrena el modelo final con los mejores hiperparámetros encontrados
best_params = study.best_params
best_params['objective'] = 'regression'
best_params['metric'] = []

model_reg = lgb.train(
    best_params,
    train_data,
    num_boost_round=50000,
    valid_sets=val_data_list,
    valid_names=[f'validation_{p}' for p in periodos_valid],
    feval=mape_sum_lgb,
    callbacks=[
        lgb.early_stopping(stopping_rounds=1000, first_metric_only=True),
        lgb.log_evaluation(period=50)
    ]
)

os.makedirs('./modelos', exist_ok=True)
model_reg.save_model('./modelos/lgbm_model_reg.txt')

[I 2025-06-11 00:55:32,532] A new study created in memory with name: no-name-96ce2dcf-0dcf-446b-bb1a-e8b9a39e9b25


Training until validation scores don't improve for 50 rounds


In [ ]:
# Obtener la importancia de cada variable
importancia = model_reg.feature_importance(importance_type='gain')
nombres = X_train.columns

# Crear un DataFrame ordenado por importancia
df_importancia = pd.DataFrame({'feature': nombres, 'importance': importancia})
df_importancia = df_importancia.sort_values(by='importance', ascending=False)

# Mostrar las variables más importantes
print(df_importancia)

# Si quieres visualizarlo gráficamente:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
plt.barh(df_importancia['feature'], df_importancia['importance'])
plt.gca().invert_yaxis()
plt.title('Importancia de variables LightGBM')
plt.xlabel('Importancia')
plt.show()

In [ ]:
# --- PREDICCIÓN Y EVALUACIÓN por periodo ---
for i, (X_val, y_val, periodo) in enumerate(zip(X_val_list, y_val_list, periodos_valid)):
    proba_no_cero = model_clasif.predict(X_val)
    umbral = 0.25
    pred_bin = (proba_no_cero > umbral)
    pred_reg = np.zeros(len(X_val))
    if pred_bin.sum() > 0:
        pred_reg[pred_bin] = model_reg.predict(X_val[pred_bin])
    y_val_real = y_val.values
    # WAPE solo en no-cero
    mask_nocero = y_val_real != 0
    if mask_nocero.sum() > 0:
        wape_nocero = np.sum(np.abs(y_val_real[mask_nocero] - pred_reg[mask_nocero])) / np.sum(np.abs(y_val_real[mask_nocero]))
        print(f"WAPE (no-cero) periodo {periodo}: {wape_nocero:.4f}")
    else:
        print(f"WAPE (no-cero) periodo {periodo}: N/A (no hay valores no-cero)")
    # También puedes seguir mostrando el WAPE global
    wape = np.sum(np.abs(y_val_real - pred_reg)) / np.sum(np.abs(y_val_real))
    print(f"WAPE global periodo {periodo}: {wape:.4f}")
    print(f"Valores distintos de cero en pred_reg: {(pred_reg != 0).sum()} de {len(pred_reg)}")

In [ ]:
# Armar listado de las 100 predicioones con mayor diferencia entre predicción y valor real con product_id, customer_id 
df_val = X_val_list[0].copy()
df_val['CLASE_REAL'] = y_val_list[0].values
df_val['CLASE_PRED'] = pred_reg
df_val['DIF'] = np.abs(df_val['CLASE_REAL'] - df_val['CLASE_PRED'])
df_val = df_val.sort_values(by='DIF', ascending=False).head(50)
df_val[['PRODUCT_ID', 'CUSTOMER_ID', 'CLASE_REAL', 'CLASE_PRED', 'DIF']]
# Agrupar las diferencias por PRODUCT_ID


In [ ]:
df_dif_product = df_val.groupby('PRODUCT_ID')['DIF'].sum().reset_index()
print("Diferencias promedio por PRODUCT_ID:")   
df_dif_product.sort_values(by='DIF', ascending=False).head(50)


In [ ]:
# Agrupar las diferencias por CUSTOMER_ID
df_dif_customer = df_val.groupby('CUSTOMER_ID')['DIF'].sum().reset_index()
print("Diferencias promedio por CUSTOMER_ID:")
df_dif_customer.sort_values(by='DIF', ascending=False).head(50)



In [ ]:
# Cargo los datos sobre los que quiero hacer predicciones

conn =  cx_Oracle.connect(user="pc",password="p201404",dsn="siatchdesa")
query = "select * from L_VM_COMPLETA where periodo = 201912" 
df_pred = pd.read_sql(query, conn, chunksize=1000000)  # Lee en chunks para no llenar la RAM

# Para concatenar todos los chunks en un solo DataFrame (si tienes suficiente RAM)
df_pred_full = pd.concat(df_pred, ignore_index=True)
conn.close()

In [ ]:
# Convertir las variables categóricas a tipo 'category'
for col in categorical_features:
    df_pred_full[col] = df_pred_full[col].astype('category')

# Con el modelo entrenado, hacemos predicciones 
X_pred = df_pred_full[['PERIODO','CUSTOMER_ID','PRODUCT_ID',
'TN_DELTA_01','TN_DELTA_02','TN_DELTA_03','TN_DELTA_06','TN_DELTA_12']]


In [ ]:

predictions = model.predict(X_pred)
# Agregar las predicciones al DataFrame original
df_pred_full['PREDICCIONES'] = predictions
# Imprimir las primeras filas del DataFrame con las predicciones
print(df_pred_full.head())
# Guardar el DataFrame con las predicciones en un archivo CSV
df_pred_full.to_csv('predicciones.csv', index=False)
# Imprimir el número de filas y columnas del DataFrame con las predicciones
print(f"Número de filas: {df_pred_full.shape[0]}, Número de columnas: {df_pred_full.shape[1]} con predicciones.")



In [ ]:
# Con el DataFrame de predicción, actualizamos la base de datos
# el criterio es actualizar la tabla L_VM_COMPLETA_PREDICCIONES con las nuevas predicciones
# la columnna PREDICCIONES se debe actualizar con los nuevos valores
# la clave primaria es (PERIODO, CUSTOMER_ID, PRODUCT_ID)
# Hacer commit cada 10000 filas para evitar problemas de memoria

# Conectar a la base de datos para actualizar los datos de predicción
conn = cx_Oracle.connect(user="pc", password="p201404", dsn="siatchdesa")
# Crear un cursor para ejecutar las actualizaciones
cursor = conn.cursor()
update_query = """
    UPDATE L_VM_COMPLETA_PREDICCIONES
    SET PREDICCION = NULL
"""
cursor.execute(update_query)
# Hacer commit para aplicar el cambio de NULL
conn.commit()


In [ ]:


# Imprimir mensaje de inicio de actualización
print("Iniciando actualización de la tabla L_VM_COMPLETA_PREDICCIONES con las nuevas predicciones.")

# Iterar sobre las filas del DataFrame con las predicciones
for index, row in df_pred_full.iterrows():
    periodo = row['PERIODO']
    customer_id = row['CUSTOMER_ID']
    product_id = row['PRODUCT_ID']
    prediccion = row['TN'] + row['PREDICCIONES']
   # prediccion = row['PREDICCIONES']
    
    # Actualizar la tabla L_DATOS_PREDICCION con la nueva predicción
    update_query = """
        UPDATE L_VM_COMPLETA_PREDICCIONES
        SET PREDICCION = :prediccion
        WHERE PERIODO = :periodo AND CUSTOMER_ID = :customer_id AND PRODUCT_ID = :product_id
    """
    cursor.execute(update_query, {'prediccion': prediccion, 'periodo': periodo, 'customer_id': customer_id, 'product_id': product_id})  
    # Hacer commit cada 10000 filas para evitar problemas de memoria
    if index % 10000 == 0:
        conn.commit()
        print(f"Actualizadas {index} filas de L_VM_COMPLETA_PREDICCIONES con las nuevas predicciones.")
# Confirmar los cambios en la base de datos
conn.commit()
# Cerrar el cursor y la conexión
cursor.close()
conn.close()
# Imprimir mensaje de finalización
print("Actualización de la tabla  completada con las nuevas predicciones.")
